# Тестовое задание SMLab

## Задание 1. SQL.

Дана таблица с позициями чеков checks и с каталогом cat. Посчитать количество чеков, в которых куплено больше 2-х пар любой обуви и есть любой товар из категории туризм.
Написать одним запросом на Oracle SQL.

In [1]:
import sqlite3
import pandas as pd

In [2]:
# создаём (или открываем) базу
conn = sqlite3.connect("test_SM.db")
cur = conn.cursor()

# создаём таблицу cat
cur.execute("""
    CREATE TABLE cat (
        art VARCHAR2(10) PRIMARY KEY,
        product VARCHAR2(30),
        category VARCHAR2(30))
    """)

# создаём таблицу checks
cur.execute("""
    CREATE TABLE checks (
        id_check NUMBER(10),
        id_pos NUMBER(10),
        art VARCHAR2(10),
        quantity NUMBER(10),
        CONSTRAINT PK_KEY_CHECKS PRIMARY KEY (id_check, id_pos),
        CONSTRAINT FK_KEY_CHECKS FOREIGN KEY (art) REFERENCES checks(art))
    """)

# данные для cat
cat_data = [
    ["A1", "кроссовки", "обувь"],
    ["A2", "кеды", "обувь"],
    ["A3", "сланцы", "обувь"],
    ["A4", "куртка", "одежда"],
    ["A5", "ветровка", "одежда"],
    ["A6", "шорты", "одежда"],
    ["A7", "палатка", "туризм"],
    ["A8", "коврик", "туризм"],
    ["A9", "спальник", "туризм"],
    ["A10", "рюкзак", "туризм"],
]

# данные для checks
checks_data = [
    [1, 1, "A1", 1],
    [1, 2, "A4", 3],
    [1, 3, "A2", 2],
    [1, 4, "A10", 1],
    [1, 5, "A8", 4],
    [2, 1, "A5", 1],
    [2, 2, "A7", 1],
    [2, 3, "A3", 2],
    [3, 1, "A2", 1],
    [3, 2, "A9", 1],
    [3, 3, "A5", 2],
    [3, 4, "A6", 1],
    [3, 5, "A3", 3],
    [4, 1, "A1", 3],
    [4, 2, "A2", 1],
    [5, 1, "A2", 1],
    [5, 2, "A8", 5],
    [5, 3, "A3", 1],
    [5, 4, "A4", 1],
    [5, 5, "A5", 1],
]

# вставляем данные
cur.executemany('INSERT OR REPLACE INTO checks VALUES (?, ?, ?, ?)', checks_data)
cur.executemany('INSERT OR REPLACE INTO cat VALUES (?, ?, ?)', cat_data)

conn.commit()
conn.close()

In [3]:
text = """
    SELECT COUNT(*) AS checks_cnt
    FROM (
        SELECT ch.id_check AS id_check
        FROM checks ch JOIN cat c ON ch.art = c.art
        GROUP BY ch.id_check
        HAVING SUM(CASE WHEN c.category = "обувь" THEN ch.quantity ELSE 0 END) > 2 AND
            SUM(CASE WHEN c.category = "туризм" THEN 1 ELSE 0 END) > 0
    )
"""
conn = sqlite3.connect("test_SM.db")
cur = conn.cursor()

cur.execute(text)
res = cur.fetchall()
res = pd.DataFrame(res)
res.columns = ["checks_cnt"]
res

,checks_cnt
0,2
